In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [8]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t3pomwsb
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t3pomwsb
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [9]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Reduction

In [10]:
!nvcc -o bin/reduction_global reduction/reduction_global_kernel.cu reduction/reduction_global.cpp

In [12]:
!nvcc -o bin/reduction_shared reduction/reduction_shared_kernel.cu reduction/reduction_shared.cpp

In [13]:
!nvprof bin/reduction_global

==3445== NVPROF is profiling process 3445, command: bin/reduction_global
Time= 14.905 msec, bandwidth= 4.502295 GB/s
host: 0.996007, device 0.996007
==3445== Profiling application: bin/reduction_global
==3445== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.26%  1.41899s      2424  585.39us  451.92us  791.44us  global_reduction_kernel(float*, float*, int, int)
                    3.70%  55.115ms       100  551.15us  544.79us  553.27us  [CUDA memcpy DtoD]
                    1.04%  15.540ms         1  15.540ms  15.540ms  15.540ms  [CUDA memcpy HtoD]
                    0.00%  1.7920us         1  1.7920us  1.7920us  1.7920us  [CUDA memcpy DtoH]
      API calls:   78.32%  1.46460s       101  14.501ms  10.840us  34.901ms  cudaDeviceSynchronize
                   19.37%  362.27ms         2  181.13ms  267.71us  362.00ms  cudaMalloc
                    1.15%  21.413ms       102  209.94us  22.889us  16.156ms  cudaMemcpy

In [14]:
!nvprof bin/reduction_shared

==3482== NVPROF is profiling process 3482, command: bin/reduction_shared
Time= 2.565 msec, bandwidth= 26.165136 GB/s
host: 0.996007, device 0.996007
==3482== Profiling application: bin/reduction_shared
==3482== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   74.34%  201.45ms       303  664.86us  4.2880us  2.5121ms  reduction_kernel(float*, float*, unsigned int)
                   20.15%  54.600ms       101  540.59us  537.68us  542.77us  [CUDA memcpy DtoD]
                    5.51%  14.920ms         1  14.920ms  14.920ms  14.920ms  [CUDA memcpy HtoD]
                    0.00%  1.6640us         1  1.6640us  1.6640us  1.6640us  [CUDA memcpy DtoH]
      API calls:   58.41%  253.49ms         1  253.49ms  253.49ms  253.49ms  cudaDeviceSynchronize
                   37.06%  160.82ms         2  80.410ms  201.04us  160.62ms  cudaMalloc
                    3.71%  16.084ms       103  156.16us  5.0930us  15.104ms  cudaMemcpy
  

### Wnioski

1. Widzimy, że redukcja wykorzystująca pamięć współdzieloną wykonuję się ~7 razy szybciej na tych danych (total time).
2. Widzimy, że w przypadku redukcji wykorzystującej pamięć współdzieloną czas spędzony na GPU jest istotnie niższy (?) i głównie spędzony na synchornizacji oraz alokacji pamięci.
3. Ponownie w przypadku redukcji wykorzystującej pamięć współdzieloną wykorzystujemy znacznie więcej przepustowości (więcej przesyłów danych) (logiczne).



## Warp Divergence

In [15]:
!nvcc -o bin/warp_divergence_interleaving warp-divergence/reduction.cpp warp-divergence/reduction_kernel_interleaving.cu -I include/

In [16]:
!nvcc -o bin/warp_divergence_sequential warp-divergence/reduction.cpp warp-divergence/reduction_kernel_sequential.cu -I include/

In [17]:
!nvprof bin/warp_divergence_interleaving

==5668== NVPROF is profiling process 5668, command: bin/warp_divergence_interleaving
Time= 3.073 msec, bandwidth= 21.836803 GB/s
host: 0.996007, device 0.996007
==5668== Profiling application: bin/warp_divergence_interleaving
==5668== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.44%  15.580ms         1  15.580ms  15.580ms  15.580ms  [CUDA memcpy HtoD]
                   40.78%  12.597ms        16  787.29us  6.4310us  2.0822ms  reduction_kernel_1(float*, float*, unsigned int)
                    8.77%  2.7080ms         5  541.59us  541.07us  541.94us  [CUDA memcpy DtoD]
                    0.01%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   90.45%  312.63ms         2  156.32ms  257.44us  312.37ms  cudaMalloc
                    4.60%  15.883ms         7  2.2690ms  7.6500us  15.778ms  cudaMemcpy
                    4.39%  15.181ms         1  15.181ms  15.181ms  15.181ms

In [18]:
!nvprof bin/warp_divergence_sequential

==5735== NVPROF is profiling process 5735, command: bin/warp_divergence_sequential
Time= 2.590 msec, bandwidth= 25.914761 GB/s
host: 0.996007, device 0.996007
==5735== Profiling application: bin/warp_divergence_sequential
==5735== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   52.71%  14.350ms         1  14.350ms  14.350ms  14.350ms  [CUDA memcpy HtoD]
                   37.33%  10.165ms        16  635.29us  6.2070us  1.6787ms  reduction_kernel_2(float*, float*, unsigned int)
                    9.95%  2.7088ms         5  541.76us  541.14us  542.96us  [CUDA memcpy DtoD]
                    0.01%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   84.50%  164.43ms         2  82.215ms  153.67us  164.28ms  cudaMalloc
                    7.51%  14.610ms         7  2.0872ms  5.2630us  14.525ms  cudaMemcpy
                    6.58%  12.811ms         1  12.811ms  12.811ms  12.811ms  cu

### Wnioski

Czasy oraz charakterystyki (pamięć, procesor) są podobne. Również wykorzystanie zasobów (na jakich operacjach ile) jest podobne.

## Loop unrolling

In [20]:
!nvcc -o bin/loop_unrolling_cg loop-unrolling/reduction.cpp loop-unrolling/reduction_cg_kernel.cu -I include/

In [21]:
!nvcc -o bin/loop_unrolling_wp loop-unrolling/reduction.cpp loop-unrolling/reduction_wp_kernel.cu -I include/

In [22]:
!nvprof bin/loop_unrolling_cg

==7294== NVPROF is profiling process 7294, command: bin/loop_unrolling_cg
Time= 0.824 msec, bandwidth= 81.480370 GB/s
host: 0.996007, device 0.996007
==7294== Profiling application: bin/loop_unrolling_cg
==7294== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   55.84%  54.177ms       100  541.77us  540.91us  542.67us  [CUDA memcpy DtoD]
                   28.66%  27.803ms       202  137.64us  5.9200us  272.25us  reduction_kernel(float*, float*, unsigned int)
                   15.50%  15.034ms         1  15.034ms  15.034ms  15.034ms  [CUDA memcpy HtoD]
                    0.00%  2.1440us         1  2.1440us  2.1440us  2.1440us  [CUDA memcpy DtoH]
      API calls:   64.25%  179.98ms         2  89.989ms  161.01us  179.82ms  cudaMalloc
                   28.79%  80.647ms         1  80.647ms  80.647ms  80.647ms  cudaDeviceSynchronize
                    5.67%  15.894ms       102  155.82us  5.0010us  15.243ms  cudaMemcpy


In [23]:
!nvprof bin/loop_unrolling_wp

==7349== NVPROF is profiling process 7349, command: bin/loop_unrolling_wp
Time= 0.822 msec, bandwidth= 81.632019 GB/s
host: 0.996007, device 0.996007
==7349== Profiling application: bin/loop_unrolling_wp
==7349== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.31%  54.175ms       100  541.75us  540.98us  542.51us  [CUDA memcpy DtoD]
                   28.70%  27.615ms       202  136.71us  5.5990us  278.43us  reduction_kernel(float*, float*, unsigned int)
                   14.98%  14.414ms         1  14.414ms  14.414ms  14.414ms  [CUDA memcpy HtoD]
                    0.00%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   63.41%  170.62ms         2  85.311ms  138.16us  170.48ms  cudaMalloc
                   29.94%  80.550ms         1  80.550ms  80.550ms  80.550ms  cudaDeviceSynchronize
                    5.69%  15.308ms       102  150.07us  4.6900us  14.684ms  cudaMemcpy


### Wnioski

Oba podejścia dają właściwie te same czasy i te same charakterystyki.

## Atomic operations

In [24]:
!nvcc -o bin/atomic_simple atomic-operations/reduction.cpp atomic-operations/reduction_kernel.cu -I include/

In [25]:
!nvcc -o bin/atomic_blk atomic-operations/reduction.cpp atomic-operations/reduction_blk_atmc_kernel.cu -I include/

In [26]:
!nvcc -o bin/atomic_wrp atomic-operations/reduction.cpp atomic-operations/reduction_wrp_atmc_kernel.cu -I include/

In [27]:
!nvprof bin/atomic_simple

==7985== NVPROF is profiling process 7985, command: bin/atomic_simple
Time= 35.468 msec, bandwidth= 1.892090 GB/s
host: 0.996007, device 0.995981
==7985== Profiling application: bin/atomic_simple
==7985== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   98.07%  3.49211s       101  34.575ms  33.223ms  58.871ms  atomic_reduction_kernel(float*, float*, int)
                    1.53%  54.506ms       100  545.06us  541.68us  545.84us  [CUDA memcpy DtoD]
                    0.40%  14.304ms         1  14.304ms  14.304ms  14.304ms  [CUDA memcpy HtoD]
                    0.00%  1.5360us         1  1.5360us  1.5360us  1.5360us  [CUDA memcpy DtoH]
      API calls:   95.10%  3.54577s         1  3.54577s  3.54577s  3.54577s  cudaDeviceSynchronize
                    4.43%  165.23ms         2  82.613ms  174.42us  165.05ms  cudaMalloc
                    0.41%  15.125ms       102  148.29us  4.5370us  14.507ms  cudaMemcpy
          

In [28]:
!nvprof bin/atomic_blk

==8018== NVPROF is profiling process 8018, command: bin/atomic_blk
Time= 0.814 msec, bandwidth= 82.448395 GB/s
host: 0.996007, device 0.996007
==8018== Profiling application: bin/atomic_blk
==8018== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.20%  54.201ms       100  542.01us  541.14us  544.53us  [CUDA memcpy DtoD]
                   27.89%  26.898ms       101  266.32us  257.98us  268.25us  reduction_blk_atmc_kernel(float*, float*, unsigned int)
                   15.90%  15.335ms         1  15.335ms  15.335ms  15.335ms  [CUDA memcpy HtoD]
                    0.00%  2.1440us         1  2.1440us  2.1440us  2.1440us  [CUDA memcpy DtoH]
      API calls:   60.46%  150.80ms         2  75.400ms  146.29us  150.65ms  cudaMalloc
                   32.15%  80.192ms         1  80.192ms  80.192ms  80.192ms  cudaDeviceSynchronize
                    6.47%  16.143ms       102  158.26us  4.7550us  15.506ms  cudaMemcpy
     

In [29]:
!nvprof bin/atomic_wrp

==8063== NVPROF is profiling process 8063, command: bin/atomic_wrp
Time= 0.813 msec, bandwidth= 82.526459 GB/s
host: 0.996007, device 0.996007
==8063== Profiling application: bin/atomic_wrp
==8063== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.70%  54.200ms       100  542.00us  541.14us  542.87us  [CUDA memcpy DtoD]
                   28.06%  26.816ms       101  265.51us  255.74us  268.89us  reduction_wrp_atmc_kernel(float*, float*, unsigned int)
                   15.24%  14.566ms         1  14.566ms  14.566ms  14.566ms  [CUDA memcpy HtoD]
                    0.00%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   63.64%  171.16ms         2  85.581ms  142.10us  171.02ms  cudaMalloc
                   29.83%  80.228ms         1  80.228ms  80.228ms  80.228ms  cudaDeviceSynchronize
                    5.70%  15.332ms       102  150.31us  4.3760us  14.766ms  cudaMemcpy
     

### Wnioski

Widzimy, że ograniczenie wykorzystania operacji atomowych tylko do sytuacji gdy to niezbędne pozwala zaoszczędzić sporo czasu (35 ms vs 0.8 ms).

Oba podejścia "oszczędzające operacje atomowee" bają bardzo zbliżone czasy i charakterystyki.

## Histogram

Lekko zmodyfikowałem kod źródłowy aby móc ustawiać rozmiar tablicy i liczbę słupków podczas kompilacji jako makra `C_ARRAY_SIZE` i `C_BIN_COUNT`.

In [30]:
!nvcc -o bin/histo_basic histogram/histo.cu -I include/ -DC_ARRAY_SIZE=65536 -DC_BIN_COUNT=16

In [32]:
!nvcc -o bin/histo_big histogram/histo.cu -I include/ -DC_ARRAY_SIZE=65536 -DC_BIN_COUNT=256

In [33]:
!nvprof bin/histo_basic 0

==10765== NVPROF is profiling process 10765, command: bin/histo_basic 0
Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running naive histo
bin 0: count 1
bin 1: count 2
bin 2: count 1
bin 3: count 2
bin 4: count 1
bin 5: count 2
bin 6: count 2
bin 7: count 3
bin 8: count 1
bin 9: count 2
bin 10: count 1
bin 11: count 3
bin 12: count 1
bin 13: count 2
bin 14: count 2
bin 15: count 3
==10765== Profiling application: bin/histo_basic 0
==10765== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   64.56%  26.112us         2  13.056us  1.4080us  24.704us  [CUDA memcpy HtoD]
                   30.06%  12.159us         1  12.159us  12.159us  12.159us  naive_histo(int*, int const *, int)
                    5.38%  2.1760us         1  2.1760us  2.1760us  2.1760us  [CUDA memcpy DtoH]
      API calls:   99.64%  173.08ms         2  86.540ms  5.3010us  173.07ms  cudaMalloc
                    0.0

In [35]:
!nvprof bin/histo_basic 1

==10872== NVPROF is profiling process 10872, command: bin/histo_basic 1
Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running simple histo
bin 0: count 4096
bin 1: count 4096
bin 2: count 4096
bin 3: count 4096
bin 4: count 4096
bin 5: count 4096
bin 6: count 4096
bin 7: count 4096
bin 8: count 4096
bin 9: count 4096
bin 10: count 4096
bin 11: count 4096
bin 12: count 4096
bin 13: count 4096
bin 14: count 4096
bin 15: count 4096
==10872== Profiling application: bin/histo_basic 1
==10872== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   77.29%  96.062us         1  96.062us  96.062us  96.062us  simple_histo(int*, int const *, int)
                   20.91%  25.983us         2  12.991us  1.4080us  24.575us  [CUDA memcpy HtoD]
                    1.80%  2.2400us         1  2.2400us  2.2400us  2.2400us  [CUDA memcpy DtoH]
      API calls:   99.49%  135.36ms         2  67.679ms  5.82

In [36]:
!nvprof bin/histo_big 0

==10923== NVPROF is profiling process 10923, command: bin/histo_big 0
Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running naive histo
bin 0: count 1
bin 1: count 1
bin 2: count 1
bin 3: count 1
bin 4: count 1
bin 5: count 1
bin 6: count 1
bin 7: count 1
bin 8: count 1
bin 9: count 1
bin 10: count 1
bin 11: count 2
bin 12: count 1
bin 13: count 1
bin 14: count 1
bin 15: count 1
bin 16: count 1
bin 17: count 1
bin 18: count 1
bin 19: count 1
bin 20: count 1
bin 21: count 1
bin 22: count 1
bin 23: count 1
bin 24: count 1
bin 25: count 1
bin 26: count 1
bin 27: count 1
bin 28: count 1
bin 29: count 1
bin 30: count 1
bin 31: count 1
bin 32: count 1
bin 33: count 1
bin 34: count 1
bin 35: count 1
bin 36: count 1
bin 37: count 1
bin 38: count 1
bin 39: count 1
bin 40: count 1
bin 41: count 1
bin 42: count 1
bin 43: count 1
bin 44: count 1
bin 45: count 2
bin 46: count 1
bin 47: count 2
bin 48: count 1
bin 49: count 1
bin 50: count 1
bin 51: count 2
bin

In [37]:
!nvprof bin/histo_big 1

==10968== NVPROF is profiling process 10968, command: bin/histo_big 1
Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running simple histo
bin 0: count 256
bin 1: count 256
bin 2: count 256
bin 3: count 256
bin 4: count 256
bin 5: count 256
bin 6: count 256
bin 7: count 256
bin 8: count 256
bin 9: count 256
bin 10: count 256
bin 11: count 256
bin 12: count 256
bin 13: count 256
bin 14: count 256
bin 15: count 256
bin 16: count 256
bin 17: count 256
bin 18: count 256
bin 19: count 256
bin 20: count 256
bin 21: count 256
bin 22: count 256
bin 23: count 256
bin 24: count 256
bin 25: count 256
bin 26: count 256
bin 27: count 256
bin 28: count 256
bin 29: count 256
bin 30: count 256
bin 31: count 256
bin 32: count 256
bin 33: count 256
bin 34: count 256
bin 35: count 256
bin 36: count 256
bin 37: count 256
bin 38: count 256
bin 39: count 256
bin 40: count 256
bin 41: count 256
bin 42: count 256
bin 43: count 256
bin 44: count 256
bin 45: count 256
bin 46

### Wnioski

Widzimy, że we wszystkich konfiguracjach czas spędzony na wywołaniach z interfejsu CUDA (API calls) jest właściwie w całości spędzony na alokacji pamięci.

Widzimy też, że (wbrew moim oczekiwaniom) dla większej ilości słupków (256 > 16) obliczenia są szybsze! (dla simple_histo 52 us do 96 us, dla naive_histo 9 us do 12 us).
